# 环境准备

In [ ]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git

In [1]:
%cd ChatGLM-Tuning

/output/ChatGLM-Tuning


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/huggingface/peft.git (to revision e536616888d51b453ed354a6f1e243fecb02ea08) to /tmp/pip-req-build-_5goabvu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-_5goabvu
  Running command git rev-parse -q --verify 'sha^e536616888d51b453ed354a6f1e243fecb02ea08'
  Running command git fetch -q https://github.com/huggingface/peft.git e536616888d51b453ed354a6f1e243fecb02ea08
  Running command git checkout -q e536616888d51b453ed354a6f1e243fecb02ea08
  Resolved https://github.com/huggingface/peft.git to commit e536616888d51b453ed354a6f1e243fecb02ea08
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 kB 559.7 kB/s eta 0:

# 准备数据
## cover_alpaca2jsonl.py

In [ ]:
import json
from tqdm import tqdm


def format_example(example: dict) -> dict:
    context = f"Instruction: 创建用户故事 \n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}


def main():
    with open("/openbayes/input/input0/datasets/userstory_detail.jsonl") as f:
        examples = list(f)

    with open("/output/train.jsonl", 'w') as f:
        for example in tqdm(examples, desc="formatting.."):
            f.write(json.dumps(format_example(json.loads(example))) + '\n')


if __name__ == "__main__":
    main()

## tokenize_dataset_rows.py

In [ ]:
import json
from tqdm import tqdm

import datasets
import transformers


def preprocess(tokenizer, config, example, max_seq_length):
    prompt = example["context"]
    target = example["target"]
    prompt_ids = tokenizer.encode(prompt, max_length=max_seq_length, truncation=True,return_attention_mask=False,
                add_special_tokens=False)
    target_ids = tokenizer.encode(
        target,
        max_length=max_seq_length,
        truncation=True,
        return_attention_mask=False,
        add_special_tokens=False)
    input_ids = prompt_ids + [150001, 150004] + target_ids + [150005]
    return {"input_ids": input_ids, "seq_len": len(prompt_ids)}


def read_jsonl(path, max_seq_length, skip_overlength=False):
    model_name = "/openbayes/input/input1"
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name, trust_remote_code=True)
    config = transformers.AutoConfig.from_pretrained(
        model_name, trust_remote_code=True, device_map='auto')
    with open(path, "r") as f:
        for line in tqdm(f.readlines()):
            example = json.loads(line)
            feature = preprocess(tokenizer, config, example, max_seq_length)
            if skip_overlength and len(feature["input_ids"]) > max_seq_length:
                continue
            feature["input_ids"] = feature["input_ids"][:max_seq_length]
            yield feature


def main():
    dataset = datasets.Dataset.from_generator(
        lambda: read_jsonl("/output/train.jsonl", 384, False)
    )
    dataset.save_to_disk("/output/train")


if __name__ == "__main__":
    main()

# 训练

## 训练前推理

In [ ]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = AutoModel.from_pretrained("/openbayes/input/input1", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)

In [ ]:
def format_example(example: dict) -> dict:
    context = f"Instruction: 创建用户故事\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

In [ ]:
import json

with open("/openbayes/input/input0/datasets/userstory_detail.jsonl") as f:
    examples = list(f)

with torch.no_grad():
    idx = 0
    for example in examples[:5]:
        item = json.loads(example)
        feature = format_example(item)
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        inputs = model.prepare_inputs_for_generation(input_ids)
        for k,v in inputs.items():
            if v is not None:
                inputs[k] = v.to("cuda")
        outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
        out = outputs[0].tolist()[input_ids.size()[-1]:]
        answer = tokenizer.decode(out)
        item['infer_answer'] = answer
        print(input_text)
        print(answer)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        idx += 1

## 训练

In [3]:
!sed -i "s/THUDM\/chatglm-6b/\/openbayes\/input\/input1/" finetune.py
!cat finetune.py
!python finetune.py \
    --dataset_path /output/train \
    --lora_rank 8 \
    --per_device_train_batch_size 6 \
    --gradient_accumulation_steps 1 \
    --max_steps 3000 \
    --save_steps 1000 \
    --save_total_limit 2 \
    --learning_rate 1e-4 \
    --fp16 \
    --remove_unused_columns false \
    --logging_steps 50 \
    --output_dir /output/lora

from transformers.integrations import TensorBoardCallback
from torch.utils.tensorboard import SummaryWriter
from transformers import TrainingArguments
from transformers import Trainer, HfArgumentParser
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType
from dataclasses import dataclass, field
import datasets
import os


tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)


@dataclass
class FinetuneArguments:
    dataset_path: str = field(default="data/alpaca")
    model_path: str = field(default="output")
    lora_rank: int = field(default=8)


class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    labels_list = []
    for ids_l, feature i

# 训练后推理

In [4]:
import torch

from transformers import AutoModel
from transformers import AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("/openbayes/input/input1", trust_remote_code=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("/openbayes/input/input1", trust_remote_code=True)


peft_path = "/output/lora/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//10.111.0.1'), PosixPath('tcp'), PosixPath('443')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//alchemist-experience'), PosixPath('7890'), PosixPath('http')}
  warn(msg)
/usr/loc


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/usr/local/lib/python3.8/site-packages/peft/tuners/lora.py:191: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


## 原输入测试

In [5]:
import json

def format_example(example: dict) -> dict:
    context = f"Instruction: create user story tasks \n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}

with open("/openbayes/input/input0/datasets/userstory_detail.jsonl") as f:
    examples = list(f)

with torch.no_grad():
    idx = 0
    for example in examples[:5]:
        item = json.loads(example)
        feature = format_example(item)
        input_text = feature['context']
        input_ids = tokenizer.encode(input_text, return_tensors="pt")
        inputs = model.prepare_inputs_for_generation(input_ids)
        for k,v in inputs.items():
            if v is not None:
                inputs[k] = v.to("cuda")
        outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
        out = outputs[0].tolist()[input_ids.size()[-1]:]
        answer = tokenizer.decode(out)
        item['infer_answer'] = answer
        print(input_text)
        print(answer)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        idx += 1

Instruction: create user story tasks 
Input:  Animation and Comics:Browse and search for animations and comics
Answer: 

用户故事:可以浏览和搜索动画和漫画
作为一个动画和漫画爱好者
我想在动画和漫画应用中浏览和搜索动画和漫画
以便于我能够找到我最喜欢的动画和漫画

AC 1: 动画和漫画爱好者可以在动画和漫画应用中浏览和搜索动画和漫画
假设 用户已经登录了动画和漫画应用
当 用户点击浏览和搜索按钮
于是 用户可以看到所有动画和漫画的列表,并可以搜索特定的动画和漫画
### 1.Answer:
 
用户故事：可以浏览和搜索动画和漫画
作为一个 Animation and Comics 应用的用户
我想要浏览和搜索动画和漫画
以便于我能够找到我喜欢的动画和漫画

AC 1: 用户可以浏览和搜索动画和漫画
假设 用户打开 Animation and Comics 应用
当 用户点击浏览和搜索动画和漫画按钮
于是 用户可以看到动画和漫画的列表，并可以搜索特定的动画和漫画 


Instruction: create user story tasks 
Input:  Animation and Comics:Participate in online forums and discussions
Answer: 

用户故事:参与在线论坛和讨论
作为一个Animation and Comics应用的用户
我想参与在线论坛和讨论
以便于我可以与其他用户分享我的观点和经验,并且可以获取其他用户的观点和经验。

AC 1: 用户可以参与在线论坛和讨论
假设 用户已经登录Animation and Comics应用
当 用户点击参与在线论坛和讨论按钮
于是 用户可以参与在线论坛和讨论
### 2.Answer:
 
用户故事：可以参与在线论坛和讨论
作为一个 Animation and Comics 应用的用户
我想参与在线论坛和讨论
以便于我可以与其他爱好者分享我的想法，观点和经验。

AC 1: 用户可以参与在线论坛和讨论
假设 用户已经登录到 Animation and Comics 应用
当 用户点击论坛按钮
于是 用户可以参与在线论坛和讨论 


In

## 任意Prompt推理

In [6]:
def evaluate(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors="pt")
    inputs = model.prepare_inputs_for_generation(input_ids)
    for k,v in inputs.items():
        if v is not None:
            inputs[k] = v.to("cuda")
    outputs = model.generate(**inputs, max_length=512, eos_token_id=tokenizer.eop_token_id)
    out = outputs[0].tolist()[input_ids.size()[-1]:]
    answer = tokenizer.decode(out)

    print(input_text)
    print(answer)

In [19]:
evaluate("Instruction: create Agile user story for following topic \nInput: 美团外卖：注册外卖骑士")

Instruction: create Agile user story for following topic 
Input: 美团外卖：注册外卖骑士
美团外卖:注册外卖骑士

作为 美团外卖的用户
我想 注册一个外卖骑士
以便于 我可以使用外卖骑士的功能

AC 1: 美团外卖的用户可以使用注册表单注册外卖骑士
假设 用户输入了注册表单中的所有信息
当 用户提交注册表单
于是 用户可以注册一个外卖骑士

AC 2: 美团外卖的用户可以使用手机号码注册外卖骑士
假设 用户输入了手机号码
当 用户提交注册表单
于是 用户可以注册一个外卖骑士

AC 3: 美团外卖的用户可以使用邮箱地址注册外卖骑士
假设 用户输入了邮箱地址
当 用户提交注册表单
于是 用户可以注册一个外卖骑士


In [29]:
evaluate("Instruction: 创建用户故事\nInput: 电影网站: 查看订单详情 \nAnswer:")

Instruction: 创建用户故事
Input: 电影网站: 查看订单详情 
Answer:

用户故事:可以查看订单详情
作为一个电影网站的用户
我想查看订单详情
以便于我可以了解订单的详细信息,如订单号、订单状态、订单内容等。

AC 1: 用户可以查看订单详情
假设 用户已经登录
当 用户点击查看订单详情按钮
于是 用户可以查看订单详情,包括订单号、订单状态、订单内容等。

AC 2: 用户可以查看订单详情
假设 用户没有登录
当 用户点击查看订单详情按钮
于是 用户被要求先登录,然后才能查看订单详情,包括订单号、订单状态、订单内容等。


In [9]:
evaluate("Instruction: 创建用户故事\nInput: 团购网站:用户注册\nAnswer:")

Instruction: 创建用户故事
Input: 团购网站:用户注册
Answer:
团购网站的用户注册故事:

作为一个团购网站的用户

我想注册一个团购网站的用户账号

以便于我可以使用团购服务,如购买商品、服务等。

AC 1: 团购网站的用户注册账号
假设 用户输入了注册信息
当 用户提交注册信息
于是 用户可以成功注册一个团购网站的用户账号

AC 2: 团购网站的用户注册账号
假设 用户输入了错误的注册信息
当 用户提交注册信息
于是 用户可以被提示输入正确的注册信息,并重新提交注册信息


In [10]:
evaluate("Instruction: 创建用户故事\nInput: 博客网站:发表新文章\nAnswer:")

Instruction: 创建用户故事
Input: 博客网站:发表新文章
Answer:


用户故事:可以在博客网站上发表新文章
作为一个博客作者
我想在博客网站上发表新文章
以便于我可以发布最新的博客内容,并且可以吸引更多的读者。

AC 1: 博客作者可以在博客网站上发表新文章
假设 博客作者已经登录博客网站
当 博客作者点击发表新文章按钮
于是 博客作者可以在博客网站上发表新文章


In [26]:
evaluate("Instruction: 创建用户故事\nInput: 京西商城：填写订单信息 \nAnswer:")

Instruction: 创建用户故事
Input: 京西商城：填写订单信息 
Answer:
用户故事:可以在京西商城填写订单信息
作为一个京西商城的用户
我想在购物时填写订单信息
以便于我能够完成订单

AC 1: 用户可以在京西商城填写订单信息
假设 用户已经选择了商品
当 用户点击提交订单按钮
于是 用户可以在订单信息表单中填写订单信息,并提交订单


In [ ]:
evaluate("Instruction: 创建用户故事 \nInput: 巨信聊天：发送文本信息\nAnswer:")

Instruction: 创建用户故事 
Input: 巨信聊天：发送文本信息
Answer:

作为一个巨信聊天应用的用户
我想要发送文本信息
以便于我能够与朋友进行有效的沟通

AC 1: 用户可以发送文本信息
假设 用户已经登录巨信聊天应用
当 用户点击发送按钮
于是 用户可以选择发送文本信息


In [ ]:
evaluate("Instruction: 创建用户故事 \nInput: 巨信聊天：发送文本信息\nAnswer:")